# Megaline Prepaid Plan Report
Find here a review of Megaline prepaid plans (both Ultimate and Surf), customers and revenue as requested by the commercial department.  I am provided 5 csv files to review including 2018 plan information, user activity and revenue for 500 customers, a sufficiently large sample.  

The purpose of this review is to test the following hypotheses:
 1. The average revenue from users of Ultimate and Surf calling plans differs.
 2. The average revenue from users in NY and NJ area is different from that of the users form other regions.

Data on user behavior is stored in the files `/datasets/megaline_calls.csv`, `/datasets/megaline_internet.csv`, `/datasets/megaline_messages.csv`, `/datasets/megaline_plans.csv` and `/datasets/megaline_users.csv`. 
 
This report will consist of these three general stages:
 1. Data overview & preprocessing
 2. Data analysis
 3. Hypotheses Testing

# Data Overview & Preprocessing

## Initialization

Several Python libraries will be necessary for this project. All are loaded in the first cell of the following code.

In [1]:
# Loading all the libraries
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import statistics as st
from scipy import stats as ss
import calendar

# Load the data files into different DataFrames
calls_df = pd.read_csv('../megaline_calls.csv')
internet_df = pd.read_csv('../megaline_internet.csv')
messages_df = pd.read_csv('../megaline_messages.csv')
plans_df = pd.read_csv('../megaline_plans.csv')
users_df = pd.read_csv('../megaline_users.csv')

##### **Plan of Action:**

There are five given tables with basic information read in each holds a different piece of user/activity information.  Megaline user and subscription info is in the 'megaline_users.csv' file. Plan details are recorded in the 'megaline_plans.csv' file.  The other three tables hold information about specific calls, texts, and web sessions that were completed on the Megaline Telecom System.

Most of the column names are reasonable.  I will be editing a few for ease of organization (adding descriptors to the 'id' columns) and others for ease of calling (using abbreviations and otherwise shortening the column names).

I will confirm that the data is sufficient to test the hypotheses; however, if there are inappropriate data types and a few missing values, that will have to be addressed during pre-processing.

Pre-processing:
Correct the formatting in the column headers and deal with the missing values. Then, check whether there are duplicates in the data. Find individual users' pertinent monthly stats: number of calls and minutes, number of texts, volume of data, revenue to company.

Header style and data types:

Change column names according to the rules of good style:
* If the name has several words, use snake_case
* All characters must be lowercase
* Delete spaces

Update above-referenced data-type changes:
* Unique ID's should be object type
* Currency should be float
* Dates should be datetime



## Plans

In [2]:
# Print the general/summary information about the plans' DataFrame

display(plans_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   messages_included      2 non-null      int64  
 1   mb_per_month_included  2 non-null      int64  
 2   minutes_included       2 non-null      int64  
 3   usd_monthly_pay        2 non-null      int64  
 4   usd_per_gb             2 non-null      int64  
 5   usd_per_message        2 non-null      float64
 6   usd_per_minute         2 non-null      float64
 7   plan_name              2 non-null      object 
dtypes: float64(2), int64(5), object(1)
memory usage: 256.0+ bytes


None

## Load data

Five csv files were provided by the commercial department. All are loaded and assigned variable names and reviewed in the next cells

### Observations
* Column descriptions:
  * plan_name — calling plan name
  * usd_monthly_fee — monthly charge in US dollars
  * minutes_included — monthly minute allowance
  * messages_included — monthly text allowance
  * mb_per_month_included — data volume allowance (in megabytes)
  * usd_per_minute — price per minute after exceeding the package limits (e.g., if
  * the package includes 100 minutes, the 101st minute will be charged)
  * usd_per_message — price per text after exceeding the package limits
  * usd_per_gb — price per extra gigabyte of data after exceeding the package
  * limits (1 GB = 1024 megabytes
* Column name updates needed:
  * 'messages_included' to 'msgs_incl'
  * 'mb_per_month_included' to 'mb_incl'
  * 'minutes_included' to 'min_incl'
  * 'usd_monthly_pay' to 'plan_usd'
  * 'usd_per_gb' to 'gb_usd'
  * 'usd_per_message' to 'msg_usd'
  * 'usd_per_minute' to 'min_usd'
  * 'plan_name' to 'plan'
* Data type updates needed:
  * 'usd_monthly_pay' to float64
  * 'usd_per_gb' to float64
* No null values


### Fix data

In [3]:
#Rename columns as per above:
plans_df = plans_df.rename(columns={
    'messages_included' : 'msgs_incl',
    'mb_per_month_included' : 'mb_incl',
    'minutes_included' : 'min_incl',
    'usd_monthly_pay' : 'plan_usd',
    'usd_per_gb' : 'gb_usd',
    'usd_per_message' : 'msg_usd',
    'usd_per_minute' : 'min_usd',
    'plan_name' : 'plan'
})

#Check resulting column names:
print('Updated Headers:')
print()
print(plans_df.columns)


Updated Headers:

Index(['msgs_incl', 'mb_incl', 'min_incl', 'plan_usd', 'gb_usd', 'msg_usd',
       'min_usd', 'plan'],
      dtype='object')


In [4]:

#Update data types as per above review:
plans_df['plan_usd'] = plans_df['plan_usd'].astype('float')
plans_df['gb_usd'] = plans_df['gb_usd'].astype('float')


#Confirm resulting data types:
print('Updated Data Types:')
print()
print(plans_df.dtypes)



Updated Data Types:

msgs_incl      int64
mb_incl        int64
min_incl       int64
plan_usd     float64
gb_usd       float64
msg_usd      float64
min_usd      float64
plan          object
dtype: object


### Enrich data

**Helpful Changes**
* Columns to add:
  * gb_incl - mb_incl/1024 - plan charges by GB so this will be more straight forward
* Create new dataframe to preserve all of the original data
* Columns to drop:
  * mb_incl - this is just redundant information with the GB column in place

In [5]:
# Add column names and functions:
plans_df['gb_incl'] = plans_df['mb_incl']/1024
plans_df['gb_incl'] = plans_df['gb_incl'].astype('int')
plans_df.dtypes

msgs_incl      int64
mb_incl        int64
min_incl       int64
plan_usd     float64
gb_usd       float64
msg_usd      float64
min_usd      float64
plan          object
gb_incl        int64
dtype: object

In [6]:
# Create new dataframe without extranious columns:
plans = plans_df.drop(columns = 'mb_incl')
plans
plans.set_index('plan', drop=True, inplace=True)
plans


,msgs_incl,min_incl,plan_usd,gb_usd,msg_usd,min_usd,gb_incl
plan,,,,,,,
surf,50,500,20.0,10.0,0.03,0.03,15
ultimate,1000,3000,70.0,7.0,0.01,0.01,30


## Users

In [7]:
# Print the general/summary information about the users' DataFrame
display(users_df.sample(5))
print()
print()
print()
display(users_df.info())

,user_id,first_name,last_name,age,city,reg_date,plan,churn_date
153,1153,Kathaleen,Crawford,30,"Bridgeport-Stamford-Norwalk, CT MSA",2018-06-19,surf,NaN
268,1268,Britni,Bowers,33,"Fresno, CA MSA",2018-01-13,surf,NaN
130,1130,Clarence,Cunningham,27,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD MSA",2018-05-17,surf,NaN
118,1118,Deane,Ramirez,28,"Columbus, OH MSA",2018-12-08,surf,NaN
89,1089,Malena,Kelley,70,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD MSA",2018-09-03,surf,NaN





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     500 non-null    int64 
 1   first_name  500 non-null    object
 2   last_name   500 non-null    object
 3   age         500 non-null    int64 
 4   city        500 non-null    object
 5   reg_date    500 non-null    object
 6   plan        500 non-null    object
 7   churn_date  34 non-null     object
dtypes: int64(2), object(6)
memory usage: 31.4+ KB


None

### Observations
* Column descriptions:
  * user_id — unique user identifier
  * first_name — user's name
  * last_name — user's last name
  * age — user's age (years)
  * reg_date — subscription date (yyyy/mm/dd)
  * churn_date — the date the user stopped using the service (if the value is missing, the calling plan was being used when this data was generated)(yyyy/mm/dd)
  * city — user's city of residence
  * plan — calling plan name
* Column name updates needed:
  * all columns appropriately named
* Data type updates needed:
  * 'user_id' to object
  * 'reg_date' to datetime
  * 'churn_date' to datetime
* Null values:
  * 'churn_date' only column containing null values
  * null values indicate an active plan at the time of data collection
  * null values will be left alone as the column is not required for this project


### Fix Data

In [8]:
#Update data types as per above review:
users_df['user_id'] = users_df['user_id'].astype('object')
users_df['reg_date'] = pd.to_datetime(users_df['reg_date'])
users_df['churn_date'] = pd.to_datetime(users_df['churn_date'])

#Confirm resulting data types:
print('Updated Data Types:')
print()
print(users_df.dtypes)




Updated Data Types:

user_id               object
first_name            object
last_name             object
age                    int64
city                  object
reg_date      datetime64[ns]
plan                  object
churn_date    datetime64[ns]
dtype: object


In [9]:
# counting clear duplicates full row:
print('Number of duplicated entries:')
print(users_df.duplicated().sum())
print()
print()
print('Number of duplicated user ID\'s:')
print(users_df.user_id.duplicated().sum())
print()
print()
print('Number of duplicated first_name+last_name pairs:')
print(users_df.duplicated(['first_name', 'last_name']).sum())

Number of duplicated entries:
0


Number of duplicated user ID's:
0


Number of duplicated first_name+last_name pairs:
0


No meaningful duplicates are found considering 3 criteria:
  * Fully duplicated rows
  * Duplicated user ID
  * Duplicated customer name

### Enrich Data

**Helpful Changes**
* Columns to add:
  * NY_NJ - Boolean True if the customer is in NY or NJ - will need this to test the second hypothesis
* Create new dataframe to preserve all of the original data
* Columns to drop: - 
  * first_name
  * last_name
  * reg_date - not needed for the scope of this project.
  * churn_date - could separate these out for a review of the revenue from customers you kept vs lost
  * age
  * as far as I can tell, there is no use for these columns inside of the scope of this project
 
 

In [10]:
# Add column to hold True if customer lives in New York or New Jersey:

#First review city names to confirm search parameters
display(users_df.city.unique())

# Create boolean column True if user lives in NY-NJ area

users_df['ny_nj'] = users_df['city'].str.contains('NY','NJ')
display(users_df.sample(10))


array(['Atlanta-Sandy Springs-Roswell, GA MSA',
       'Seattle-Tacoma-Bellevue, WA MSA',
       'Las Vegas-Henderson-Paradise, NV MSA', 'Tulsa, OK MSA',
       'Dallas-Fort Worth-Arlington, TX MSA',
       'San Francisco-Oakland-Berkeley, CA MSA',
       'Grand Rapids-Kentwood, MI MSA',
       'Orlando-Kissimmee-Sanford, FL MSA',
       'San Jose-Sunnyvale-Santa Clara, CA MSA',
       'Cleveland-Elyria, OH MSA',
       'Chicago-Naperville-Elgin, IL-IN-WI MSA', 'Knoxville, TN MSA',
       'New York-Newark-Jersey City, NY-NJ-PA MSA', 'Pittsburgh, PA MSA',
       'Fresno, CA MSA',
       'Washington-Arlington-Alexandria, DC-VA-MD-WV MSA',
       'Indianapolis-Carmel-Anderson, IN MSA', 'Jacksonville, FL MSA',
       'Los Angeles-Long Beach-Anaheim, CA MSA',
       'Omaha-Council Bluffs, NE-IA MSA',
       'Houston-The Woodlands-Sugar Land, TX MSA',
       'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD MSA',
       'Tampa-St. Petersburg-Clearwater, FL MSA',
       'Birmingham-Hoover, AL MSA'

,user_id,first_name,last_name,age,city,reg_date,plan,churn_date,ny_nj
420,1420,Season,Hardy,27,"San Francisco-Oakland-Berkeley, CA MSA",2018-02-22,ultimate,NaT,False
93,1093,Karima,Hampton,69,"Nashville-Davidson–Murfreesboro–Franklin, TN MSA",2018-12-17,surf,NaT,False
162,1162,Mark,Farley,20,"Los Angeles-Long Beach-Anaheim, CA MSA",2018-07-12,ultimate,NaT,False
178,1178,Charlsie,Reid,53,"Cincinnati, OH-KY-IN MSA",2018-10-05,surf,NaT,False
65,1065,Sherlyn,Noel,26,"Los Angeles-Long Beach-Anaheim, CA MSA",2018-08-03,surf,NaT,False
257,1257,Gabriel,O'donnell,61,"New York-Newark-Jersey City, NY-NJ-PA MSA",2018-01-29,surf,NaT,True
374,1374,Ching,Watts,55,"Louisville/Jefferson County, KY-IN MSA",2018-02-14,surf,NaT,False
131,1131,Shane,Morrison,60,"Los Angeles-Long Beach-Anaheim, CA MSA",2018-01-07,surf,NaT,False
46,1046,Beata,Hooper,67,"Boston-Cambridge-Newton, MA-NH MSA",2018-02-19,surf,NaT,False
458,1458,Joella,Munoz,32,"Detroit-Warren-Dearborn, MI MSA",2018-07-24,ultimate,NaT,False


In [11]:
# Create new dataframe with only columns that will be needed for this project:
users = users_df.drop(columns = ['first_name', 'last_name', 'reg_date', 'churn_date', 'age', 'city'])

print(users.sample(1))

    user_id  plan  ny_nj
147    1147  surf  False


## Calls

In [12]:
# Print the general/summary information about the calls' DataFrame
#show sample entries, null counts, data types
display(calls_df.sample(5))
print()
print()
print()
display(calls_df.info())


,id,user_id,call_date,duration
121918,1430_403,1430,2018-12-31,3.42
85935,1316_207,1316,2018-07-26,10.43
68759,1247_92,1247,2018-08-02,0.00
135130,1490_230,1490,2018-08-19,0.00
2895,1010_509,1010,2018-04-21,6.28





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137735 entries, 0 to 137734
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         137735 non-null  object 
 1   user_id    137735 non-null  int64  
 2   call_date  137735 non-null  object 
 3   duration   137735 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.2+ MB


None

### Observations

* Column descriptions:
  * id — unique call identifier
  * call_date — call date
  * duration — call duration (in minutes)
  * user_id — the identifier of the user making the call
* Column name updates needed:
  * 'id' to 'call_id'
* Data type updates needed:
  * 'user_id' to object
  * 'call_date' to datetime
* No null values


### Fix data

In [13]:
#Rename columns as per above review:
calls_df = calls_df.rename(columns={
    'id':'call_id',
})

#Check resulting column names:
print('Updated Headers:')
print()
print(calls_df.columns)


print(calls_df.duration.mean())

Updated Headers:

Index(['call_id', 'user_id', 'call_date', 'duration'], dtype='object')
6.745927033796782


In [14]:
#Update data types as per above review:
calls_df['user_id'] = calls_df['user_id'].astype('object')
calls_df['call_date'] = pd.to_datetime(calls_df['call_date'])

#Confirm resulting data types:
print()
print(calls_df.dtypes)




call_id              object
user_id              object
call_date    datetime64[ns]
duration            float64
dtype: object


In [15]:
# counting clear duplicates full row:
print('Number of duplicated entries:')
print(calls_df.duplicated().sum())
print()
print()
print('Number of duplicated call ID\'s')
print(calls_df.call_id.duplicated().sum())
print()
print()

Number of duplicated entries:
0


Number of duplicated call ID's
0




### Enrich data

**Helpful Changes**
* Columns to add:
  * min_spent - rounded up call duration by call
  * month - holds the month of the call
* Create new dataframe to preserve all of the original data
* Columns to drop:
  * call_date - we have the info we need in month column
  * call_id - not required to track in the scope
  * duration - minutes spent will be the only data required going forward

In [16]:
#Adding a rounded-up call duration:
calls_df['min_spent'] = np.ceil(calls_df['duration'])
calls_df['min_spent'] = calls_df['min_spent'].astype('int')

#Adding a month of call column:
calls_df['month'] = calls_df['call_date'].dt.month

print(calls_df.sample(1))

       call_id user_id  call_date  duration  min_spent  month
8902  1041_337    1041 2018-04-11       2.6          3      4


In [17]:
# Create a new dataframe without extranious columns:
calls = calls_df.drop(columns = ['call_date', 'call_id'])
calls.columns

Index(['user_id', 'duration', 'min_spent', 'month'], dtype='object')

## Messages

In [18]:
# Print the general/summary information about the messages' DataFrame

#show sample entries, null counts, data types
display(messages_df.sample(5))
print()
print()
print()
display(messages_df.info())

,id,user_id,message_date
5442,1052_199,1052,2018-11-25
588,1006_219,1006,2018-12-07
64591,1412_173,1412,2018-07-17
12697,1082_276,1082,2018-08-31
9963,1075_218,1075,2018-11-07





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76051 entries, 0 to 76050
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            76051 non-null  object
 1   user_id       76051 non-null  int64 
 2   message_date  76051 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


None

### Observations
* Column descriptions:
  * id — unique text message identifier
  * message_date — text message date
  * user_id — the identifier of the user sending the text
* Column name updates needed:
  * 'id' to 'message_id'
* Data type updates needed:
  * 'user_id' to object
  * 'message_date' to datetime
* No null values

### Fix data

In [19]:
#Rename columns as per above:
messages_df = messages_df.rename(columns={'id':'msg_id', 'message_date':'msg_date'})

#Check resulting column names:
print('Updated Headers:')
print()
print(messages_df.columns)




Updated Headers:

Index(['msg_id', 'user_id', 'msg_date'], dtype='object')


In [20]:
#Update data types as per above review:
messages_df['msg_id'] = messages_df['msg_id'].astype('object')
messages_df['user_id'] = messages_df['user_id'].astype('object')
messages_df['msg_date'] = pd.to_datetime(messages_df['msg_date'])

#Confirm resulting data types:
print('Updated Data Types:')
print()
print(messages_df.dtypes)



Updated Data Types:

msg_id              object
user_id             object
msg_date    datetime64[ns]
dtype: object


In [21]:
# counting clear duplicates full row:
print('Number of duplicated entries:')
print(messages_df.duplicated().sum())
print()
print()
print('Number of duplicated message ID\'s:')
print(messages_df.msg_id.duplicated().sum())

Number of duplicated entries:
0


Number of duplicated message ID's:
0


### Enrich data

**Helpful Changes**
* Columns to add:
  * month - holds month text was sent
* Create new dataframe to preserve all of the original data
* Columns to drop:
  * msg_date - we have all the needed information stored in 'month' column

In [22]:
#Adding a month of text column:
messages_df['month'] = messages_df['msg_date'].dt.month

# Creating new dataframe and dropping columns:
messages = messages_df.drop(columns = ['msg_date'])
print(messages.sample(1))
print(messages.msg_id.unique())

        msg_id user_id  month
7261  1059_133    1059      8
['1000_125' '1000_160' '1000_223' ... '1497_547' '1497_558' '1497_613']


## Internet

In [23]:
# Print the general/summary information about the internet DataFrame
#show sample entries, null counts, data types
display(internet_df.sample(5))
print()
print()
print()
display(internet_df.info())


,id,user_id,session_date,mb_used
13819,1064_267,1064,2018-10-23,633.30
43974,1195_111,1195,2018-10-20,256.80
16542,1074_44,1074,2018-12-11,925.04
89294,1412_9,1412,2018-12-17,844.06
3260,1018_45,1018,2018-10-30,759.50





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104825 entries, 0 to 104824
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            104825 non-null  object 
 1   user_id       104825 non-null  int64  
 2   session_date  104825 non-null  object 
 3   mb_used       104825 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 3.2+ MB


None

### Observations
* Column desctiptions:
  * id — unique session identifier
  * mb_used — the volume of data spent during the session (in megabytes)
  * session_date — web session date
  * user_id — user identifier
* Column name updates needed:
  * 'id' to 'web_id'
  * add a month column
* Data type updates needed:
  * 'user_id' to object
  * 'session_date' to datetime
* No null values

### Fix data

In [24]:

#Rename columns as per above:
internet_df = internet_df.rename(columns={'id':'web_id'})

internet_df.columns

Index(['web_id', 'user_id', 'session_date', 'mb_used'], dtype='object')

In [25]:
#Update data types as per above review:
internet_df['web_id'] = internet_df['web_id'].astype('object')
internet_df['user_id'] = internet_df['user_id'].astype('object')
internet_df['session_date'] = pd.to_datetime(internet_df['session_date'])

#Confirm resulting data types:
print('Updated Data Types:')
print()
print(internet_df.dtypes)



Updated Data Types:

web_id                  object
user_id                 object
session_date    datetime64[ns]
mb_used                float64
dtype: object


In [26]:
## counting clear duplicates full row:
print('Number of duplicated entries:')
print(internet_df.duplicated().sum())
print()
print()
print('Number of duplicated session ID\'s:')
print(internet_df.web_id.duplicated().sum())

Number of duplicated entries:
0


Number of duplicated session ID's:
0


### Enrich data

**Helpful Changes**
* Columns to add:
  * gb_used - the plan is charged by extra GB, so this will streamline handling.
  * month - holds month internet was accessed
* Create new dataframe to preserve all of the original data
* Columns to drop:
  * session_date - we have all the needed information stored in 'month' column

In [27]:
# Add column names and formulas:
internet_df['gb_used'] = (internet_df['mb_used'])/1024
internet_df['month'] = internet_df['session_date'].dt.month
print(internet_df.sample(1))

         web_id user_id session_date  mb_used  gb_used  month
56072  1253_136    1253   2018-12-26   275.61  0.26915     12


In [28]:
# Create new dataframe without extraneous columns:
internet = internet_df.drop(columns = ['web_id','session_date', 'mb_used'])
internet.sample(1)

,user_id,gb_used,month
48970,1219,0.107861,12


# Data Analysis

## Plan of Action

* Study plan conditions
  * review Surf and Ultimate Plan conditions
  * review 'plans' dataframe to confirm it accurately reflects plan conditions
* Aggregate data per user per month
  * group and merge ll user/use data by user_id and month
  * calculate revenue per user per month
  * merge revenue dataframe with user/use dataframe
* Study user behavior
  * create bar graphs of the averages of measurable behavior by plan, month and population
  * create histograms of the sums of measurable behavior by plan, month and population
  * create box plots of the distributions of measurable behavior by plan, month and population

## Study plan conditions

* Surf monthly (USD)
  * cost - 20
  * minutes
    * included - 500 minutes
    * additional (no partials) - .03/minute
    * tallied and rounded up upon completion
  * messages
    * included - 50 texts
    * additional - .03/text
    * tallied upon completion
  * data
    * included - 15GB data
    * additional - 10/GB
    * summed and rounded up at end of month

* Ultimate monthly (USD)
  * cost - 70
  * minutes
    * included - 1000
    * additional (no partials) - .01/minute
    * tallied and **rounded up** upon **completion**
  * tests
    * included - 1000
    * additional - .01/text
    * tallied upon completion
  * data
    * included - 30GB
    * additional - 7/GB
    * summed and **rounded up** at **end of month**


In [29]:
# Print out the plan conditions and make sure they are clear for you
plans


,msgs_incl,min_incl,plan_usd,gb_usd,msg_usd,min_usd,gb_incl
plan,,,,,,,
surf,50,500,20.0,10.0,0.03,0.03,15
ultimate,1000,3000,70.0,7.0,0.01,0.01,30


## Aggregate data per user

### Group and calculate sums:

#### Calls

In [30]:
# Group calls made and the total duration by each user per month. Save the result.

# Isolate min_spent column to count and set index, rename columns
call_data = calls.pivot_table(values=['duration', 'min_spent'], index = ['user_id', 'month'], aggfunc = ['count', 'sum', 'mean'])
call_data = call_data.reset_index()

call_data.columns = ['user_id', 'month', 'ttl_calls', 'drop_me', 'sum_dur', 'sum_minspent', 'mean_dur', 'mean_minspent' ]
call_data = call_data.drop(columns = {'drop_me'})
display(call_data)


,user_id,month,ttl_calls,sum_dur,sum_minspent,mean_dur,mean_minspent
0,1000,12,16,116.83,124,7.301875,7.750000
1,1001,8,27,171.14,182,6.338519,6.740741
2,1001,9,49,297.69,315,6.075306,6.428571
3,1001,10,65,374.11,393,5.755538,6.046154
4,1001,11,64,404.59,426,6.321719,6.656250
...,...,...,...,...,...,...,...
2253,1498,12,39,324.77,339,8.327436,8.692308
2254,1499,9,41,330.37,346,8.057805,8.439024
2255,1499,10,53,363.28,385,6.854340,7.264151
2256,1499,11,45,288.56,308,6.412444,6.844444


#### Messages

In [31]:
# Calculate the number of messages sent by each user per month. Save the result.
#Isolate message information, count, index, rename columns
msg_data = messages.pivot_table(values=['msg_id'], index = ['user_id', 'month'], aggfunc = ['count'])
msg_data.columns = ['ttl_msgs']
msg_data['sum_msgspent'] = msg_data['ttl_msgs']
msg_data = msg_data.reset_index()
msg_data



,user_id,month,ttl_msgs,sum_msgspent
0,1000,12,11,11
1,1001,8,30,30
2,1001,9,44,44
3,1001,10,53,53
4,1001,11,36,36
...,...,...,...,...
1801,1496,9,21,21
1802,1496,10,18,18
1803,1496,11,13,13
1804,1496,12,11,11


#### Internet

In [32]:
# Calculate the volume of internet traffic used by each user per month. Save the result.

# Isolate the gb_used data to sum over indexes user_id and month, rename columns
web_data = internet.pivot_table(values='gb_used', index = ['user_id', 'month'], aggfunc = ['count','sum'])
web_data.columns = ['ttl_sesn', 'sum_gbused']

# Create new column for chargeable GB, Round monthly GB total up to next integer:
web_data['sum_gbspent'] = np.ceil(web_data['sum_gbused'])
web_data['sum_gbspent'] = web_data['sum_gbspent'].astype('int')
web_data = web_data.reset_index()
web_data.sample(4)

,user_id,month,ttl_sesn,sum_gbused,sum_gbspent
2174,1475,9,53,16.830840,17
287,1064,6,16,6.418193,7
1795,1388,12,39,12.951943,13
2233,1491,9,27,6.875488,7


#### User

In [33]:
# Reduce user data to only id, plan and region information 


user_data = users[['user_id', 'plan', 'ny_nj']]
user_data.sort_values('user_id')
user_data

,user_id,plan,ny_nj
0,1000,ultimate,False
1,1001,surf,False
2,1002,surf,False
3,1003,surf,False
4,1004,surf,False
...,...,...,...
495,1495,surf,True
496,1496,surf,False
497,1497,ultimate,False
498,1498,surf,True


### Combining

In [34]:

# Merge the data for calls, minutes, messages, internet based on user_id and month

# merge call and text data on user_id and month
calls_and_msg = pd.merge(call_data, msg_data, how = 'outer', on = ['user_id', 'month'])
calls_and_msg.sort_values(['user_id', 'month'])

,user_id,month,ttl_calls,sum_dur,sum_minspent,mean_dur,mean_minspent,ttl_msgs,sum_msgspent
0,1000,12,16.0,116.83,124.0,7.301875,7.750000,11.0,11.0
1,1001,8,27.0,171.14,182.0,6.338519,6.740741,30.0,30.0
2,1001,9,49.0,297.69,315.0,6.075306,6.428571,44.0,44.0
3,1001,10,65.0,374.11,393.0,5.755538,6.046154,53.0,53.0
4,1001,11,64.0,404.59,426.0,6.321719,6.656250,36.0,36.0
...,...,...,...,...,...,...,...,...,...
2253,1498,12,39.0,324.77,339.0,8.327436,8.692308,NaN,NaN
2254,1499,9,41.0,330.37,346.0,8.057805,8.439024,NaN,NaN
2255,1499,10,53.0,363.28,385.0,6.854340,7.264151,NaN,NaN
2256,1499,11,45.0,288.56,308.0,6.412444,6.844444,NaN,NaN


In [35]:

# Merge call and text dataframe with web_data on user_id and month
services_used = pd.merge(calls_and_msg, web_data, how = 'outer', on = ['user_id', 'month'])

services_used.sort_values(['user_id', 'month'])


,user_id,month,ttl_calls,sum_dur,sum_minspent,mean_dur,mean_minspent,ttl_msgs,sum_msgspent,ttl_sesn,sum_gbused,sum_gbspent
0,1000,12,16.0,116.83,124.0,7.301875,7.750000,11.0,11.0,5.0,1.856904,2.0
1,1001,8,27.0,171.14,182.0,6.338519,6.740741,30.0,30.0,25.0,6.756982,7.0
2,1001,9,49.0,297.69,315.0,6.075306,6.428571,44.0,44.0,53.0,13.002754,14.0
3,1001,10,65.0,374.11,393.0,5.755538,6.046154,53.0,53.0,56.0,21.807119,22.0
4,1001,11,64.0,404.59,426.0,6.321719,6.656250,36.0,36.0,51.0,18.070605,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2253,1498,12,39.0,324.77,339.0,8.327436,8.692308,NaN,NaN,65.0,22.595400,23.0
2254,1499,9,41.0,330.37,346.0,8.057805,8.439024,NaN,NaN,45.0,12.680430,13.0
2255,1499,10,53.0,363.28,385.0,6.854340,7.264151,NaN,NaN,61.0,19.035576,20.0
2256,1499,11,45.0,288.56,308.0,6.412444,6.844444,NaN,NaN,49.0,16.419756,17.0


In [36]:
# Add the user/plan information to the services_used dataframe
monthly_user = services_used.join(user_data.set_index('user_id'), on = ['user_id'], how = 'left', sort = True)
display(monthly_user)




,user_id,month,ttl_calls,sum_dur,sum_minspent,mean_dur,mean_minspent,ttl_msgs,sum_msgspent,ttl_sesn,sum_gbused,sum_gbspent,plan,ny_nj
0,1000,12,16.0,116.83,124.0,7.301875,7.750000,11.0,11.0,5.0,1.856904,2.0,ultimate,False
1,1001,8,27.0,171.14,182.0,6.338519,6.740741,30.0,30.0,25.0,6.756982,7.0,surf,False
2,1001,9,49.0,297.69,315.0,6.075306,6.428571,44.0,44.0,53.0,13.002754,14.0,surf,False
3,1001,10,65.0,374.11,393.0,5.755538,6.046154,53.0,53.0,56.0,21.807119,22.0,surf,False
4,1001,11,64.0,404.59,426.0,6.321719,6.656250,36.0,36.0,51.0,18.070605,19.0,surf,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2253,1498,12,39.0,324.77,339.0,8.327436,8.692308,NaN,NaN,65.0,22.595400,23.0,surf,True
2254,1499,9,41.0,330.37,346.0,8.057805,8.439024,NaN,NaN,45.0,12.680430,13.0,surf,False
2255,1499,10,53.0,363.28,385.0,6.854340,7.264151,NaN,NaN,61.0,19.035576,20.0,surf,False
2256,1499,11,45.0,288.56,308.0,6.412444,6.844444,NaN,NaN,49.0,16.419756,17.0,surf,False


In [37]:
# Check for empty cells:
print(monthly_user.isna().sum())

# Fill empty cells in dataframe with 0's.
monthly_user = monthly_user.fillna(0)

# Check my work
display(monthly_user.isna().sum())



user_id            0
month              0
ttl_calls         35
sum_dur           35
sum_minspent      35
mean_dur          35
mean_minspent     35
ttl_msgs         487
sum_msgspent     487
ttl_sesn          16
sum_gbused        16
sum_gbspent       16
plan               0
ny_nj              0
dtype: int64


user_id          0
month            0
ttl_calls        0
sum_dur          0
sum_minspent     0
mean_dur         0
mean_minspent    0
ttl_msgs         0
sum_msgspent     0
ttl_sesn         0
sum_gbused       0
sum_gbspent      0
plan             0
ny_nj            0
dtype: int64

In [38]:


# reset datatypes
monthly_user['sum_minspent'] = monthly_user['sum_minspent'].astype('int')
monthly_user['sum_msgspent'] = monthly_user['sum_msgspent'].astype('int')
monthly_user['sum_gbspent'] = monthly_user['sum_gbspent'].astype('int')
monthly_user['ttl_calls']= monthly_user['ttl_calls'].astype('int')
monthly_user['ttl_sesn'] = monthly_user['ttl_sesn'].astype('int')
monthly_user['ttl_msgs'] = monthly_user['ttl_msgs'].astype('int')



display(monthly_user.dtypes)

user_id            int64
month              int64
ttl_calls          int64
sum_dur          float64
sum_minspent       int64
mean_dur         float64
mean_minspent    float64
ttl_msgs           int64
sum_msgspent       int64
ttl_sesn           int64
sum_gbused       float64
sum_gbspent        int64
plan              object
ny_nj               bool
dtype: object

#### **Conclusions and notes**

All dataframes were combined into a single dataframe including user plan and region info.  Because minutes and GB are both rounded in preparation for the revenue calculation, there is no reason to leave these numbers as float type.  With entries changed to integers, the dataframe is cleaner in appearance.  This dataframe represents all the needed demographic and monthly use information for 500 customers.  The index has been reset to make applying a function to the dataframe a cleaner coding experience.

### Revenue calculations from new dataframe:

**Calculating Revenue:**
* In order to calculate revenue for each user, we will have to add the following things together:
  * Base price of the plan
  * Minutes used over the plan allowance multiplied by the price per additional minute (USD)
  * Messages used over the plan allowance multiplied by the price per additional message (USD)
  * GB used over the plan allowance multiplied by the price per additional GB (USD)
* This amount must be stored in a new row, 'revenue', and will represent the revenue per user per month

In [39]:
def revenue(row):
    
    # Information to pull from row
    
    min_spent = row['sum_minspent']
    gb_spent = row['sum_gbspent']
    msgs_spent = row['sum_msgspent']
    plan = row['plan']
    
    
    
    # Plan Details:
    
    # Plan limits:
    min_incl = plans.loc[plan, 'min_incl']
    msgs_incl = plans.loc[plan, 'msgs_incl']
    gb_incl = plans.loc[plan, 'gb_incl']
    
    # USD for plan and extras:
    plan_usd = plans.loc[plan, 'plan_usd']
    min_usd = plans.loc[plan, 'min_usd']
    msg_usd = plans.loc[plan, 'msg_usd']
    gb_usd = plans.loc[plan,'gb_usd']
    
    
    
    # Overage:
    
    x_min = (min_spent - min_incl)*min_usd
    x_msg = (msgs_spent - msgs_incl)*msg_usd
    x_gb = (gb_spent - gb_incl)*gb_usd
    
    rev = plan_usd + x_min.clip(0) + x_msg.clip(0) + x_gb.clip(0)
    
    return rev
    
# Apply the function and get the revenue.
monthly_user['revenue'] = monthly_user.apply(revenue, axis=1)

print('Complete monthly user data:')
monthly_user
   

    





Complete monthly user data:


,user_id,month,ttl_calls,sum_dur,sum_minspent,mean_dur,mean_minspent,ttl_msgs,sum_msgspent,ttl_sesn,sum_gbused,sum_gbspent,plan,ny_nj,revenue
0,1000,12,16,116.83,124,7.301875,7.750000,11,11,5,1.856904,2,ultimate,False,70.00
1,1001,8,27,171.14,182,6.338519,6.740741,30,30,25,6.756982,7,surf,False,20.00
2,1001,9,49,297.69,315,6.075306,6.428571,44,44,53,13.002754,14,surf,False,20.00
3,1001,10,65,374.11,393,5.755538,6.046154,53,53,56,21.807119,22,surf,False,90.09
4,1001,11,64,404.59,426,6.321719,6.656250,36,36,51,18.070605,19,surf,False,60.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2253,1498,12,39,324.77,339,8.327436,8.692308,0,0,65,22.595400,23,surf,True,100.00
2254,1499,9,41,330.37,346,8.057805,8.439024,0,0,45,12.680430,13,surf,False,20.00
2255,1499,10,53,363.28,385,6.854340,7.264151,0,0,61,19.035576,20,surf,False,70.00
2256,1499,11,45,288.56,308,6.412444,6.844444,0,0,49,16.419756,17,surf,False,40.00
